In [34]:
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
with open('illinois_early_vote_web.html') as f:
    page = f.read()

In [3]:
soup = BeautifulSoup(page, 'html5lib')

In [35]:
df = pd.DataFrame(columns = ['county_name', 'ev_location', 'ev_address', 'ev_city_state_zip', 'ev_location_type', 'ev_dates', 'ev_hours', 'ev_notes'])

In [45]:
infoContainers = soup.findAll('div', {'class': 'infoContainers'})
for c in infoContainers[1:]:
    try:
        headers = c.find('div', {'class': 'infoHeaders'}).findAll('asp:label')
        county_name = headers[0].text
        ev_location = headers[1].text
        ev_address = headers[2].find('br').previousSibling
        if len(headers[2].findAll('br')) > 1:
            ev_city_state_zip = headers[2].findAll('br')[1].nextSibling
        else:
            ev_city_state_zip = headers[2].find('br').nextSibling
        try:
            ev_data_grid = c.nextSibling.find('table').findAll('tr')[1]
            cells = ev_data_grid.findAll('td')
            ev_location_type = cells[0].text
            ev_dates = cells[1].text
            ev_hours = cells[2].text
            ev_notes = cells[3].text
        except:
            ev_location_type = None
            ev_dates = None
            ev_hours = None
            ev_notes = None

        results = {
            'county_name': county_name,
            'ev_location': ev_location,
            'ev_address': ev_address,
            'ev_city_state_zip': ev_city_state_zip,
            'ev_location_type': ev_location_type,
            'ev_dates': ev_dates,
            'ev_hours': ev_hours,
            'ev_notes': ev_notes
        }
        df = df.append(results, ignore_index = True)
    except: # No early vote sites
        pass
    
        
        

In [46]:
df.to_csv('illinois_ev_sites.csv', index = False)